# Exposure

## Imports

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pathlib import Path
import sys

from Bio.PDB import HSExposureCA, HSExposureCB, Selection, MMCIFParser
import pandas as pd

sys.path.append('../..')
from kissim.auxiliary import KlifsMoleculeLoader, PdbChainLoader, get_klifs_residues_mol2topdb
from kissim.encoding import ExposureFeature

_ColormakerRegistry()

In [4]:
from Bio.PDB import HSExposure

In [5]:
pd.set_option('display.max_rows', 1000)

## IO paths

In [6]:
path_to_kinsim = Path('.') / '..' / '..'
path_to_data = path_to_kinsim / 'examples' / 'data'
path_to_results = path_to_kinsim / 'examples' / 'results'

metadata_path = path_to_data / 'postprocessed' / 'klifs_metadata_postprocessed.csv'
sco_path = path_to_results / 'side_chain_orientation' / 'side_chain_orientations.p'

## Load metadata

In [7]:
klifs_metadata = pd.read_csv(metadata_path, index_col=0)
klifs_metadata.head()

,metadata_index,kinase,family,groups,pdb_id,chain,alternate_model,species,ligand_orthosteric_name,ligand_orthosteric_pdb_id,...,ac_helix,rmsd1,rmsd2,qualityscore,pocket,resolution,missing_residues,missing_atoms,full_ifp,code
0,2886,AAK1,NAK,Other,4wsq,B,A,Human,K-252A,KSA,...,in,0.777,2.125,8.6,EVLAEGGFAIVFLCALKRMVCKREIQIMRDLSKNIVGYIDSLILMD...,1.95,0,14,0000000000000010000001000000000000000000000000...,HUMAN/AAK1/4wsq_chainB_altA
1,10043,AAK1,NAK,Other,5l4q,A,A,Human,"~{N}-[5-(4-cyanophenyl)-1~{H}-pyrrolo[2,3-b]py...",LKB,...,in,0.780,2.137,9.7,EVLAEGGFAIVFLCALKRMVCKREIQIMRDLSKNIVGYIDSLILMD...,1.97,0,3,0000000000000010000000000000000000000000000000...,HUMAN/AAK1/5l4q_chainA_altA
2,7046,AAK1,NAK,Other,5te0,A,-,Human,methyl (3Z)-3-{[(4-{methyl[(4-methylpiperazin-...,XIN,...,in,0.776,2.120,8.8,EVLAEGGFAIVFLCALKRMVCKREIQIMRDLSKNIVGYIDSLILMD...,1.90,0,12,1000101000000010000001000000000000000000000000...,HUMAN/AAK1/5te0_chainA
3,843,ABL1,Abl,TK,2f4j,A,-,Human,CYCLOPROPANECARBOXYLIC ACID {4-[4-(4-METHYL-PI...,VX6,...,in,0.779,2.128,8.0,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,1.91,0,0,0000000000000010000001000000000000000000000000...,HUMAN/ABL1/2f4j_chainA
4,815,ABL1,Abl,TK,2g1t,A,-,Human,-,-,...,out,0.825,2.154,8.0,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,1.80,0,0,NaN,HUMAN/ABL1/2g1t_chainA


## Load chain

Example 6c83:
- Has GLY residues (per definition without CB atoms, e.g. **residue 136**)
- Has residues with missing CB atoms (e.g. **residue 141**)

I need to find out whether I can retrieve their pseudo-CB details from the `HSExposureCB` and `HSExposureCA` classes.

In [8]:
klifs_metadata_entry = klifs_metadata[klifs_metadata.pdb_id == '6c83'].squeeze()

In [9]:
# Load pdb file
pdb_id = '6c83'
parser = MMCIFParser()
structure = parser.get_structure(
    structure_id=pdb_id,
    filename=f'/home/dominique/Documents/data/kinsim/20190724_full/raw/PDB_download/{pdb_id}.cif'
)
model = structure[0]
chain = model['B']
residues = Selection.unfold_entities(entity_list=chain, target_level='R')

/home/dominique/anaconda3/envs/kinsim/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4875.
  PDBConstructionWarning)
/home/dominique/anaconda3/envs/kinsim/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4906.
  PDBConstructionWarning)


In [10]:
# Print residues in model - and compare with plain PDB file: correct!
len(residues)

248

In [11]:
residue_ids = [residue.id[1] for residue in residues]

## HSExposure

https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ

## `HSExposureCB`

In [12]:
exposures_cb = HSExposureCB(chain, 13)

In [13]:
len(exposures_cb)

242

### Missing residues after calculation

In [14]:
residue_ids_hse_cb = [r[0].id[1] for r in exposures_cb.property_list]
missing_hse_cb = set(residue_ids) - set(residue_ids_hse_cb)
print(f'Residues with missing HSExposureCB calculation: {missing_hse_cb}')

Residues with missing HSExposureCB calculation: {131, 141, 143, 144, 501, 253}


In [15]:
for r in residues:
    if r.id[1] in missing_hse_cb:
        print(r)
        print(list(r.get_atoms()))
        print()

<Residue GLU het=  resseq=131 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

<Residue LYS het=  resseq=141 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

<Residue LYS het=  resseq=143 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

<Residue PHE het=  resseq=144 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

<Residue ILE het=  resseq=253 icode= >
[<Atom N>, <Atom CA>, <Atom C>, <Atom O>]

<Residue ACP het=H_ACP resseq=501 icode= >
[<Atom PG>, <Atom O1G>, <Atom O2G>, <Atom O3G>, <Atom PB>, <Atom O1B>, <Atom O2B>, <Atom C3B>, <Atom PA>, <Atom O1A>, <Atom O2A>, <Atom O3A>, <Atom O5'>, <Atom C5'>, <Atom C4'>, <Atom O4'>, <Atom C3'>, <Atom O3'>, <Atom C2'>, <Atom O2'>, <Atom C1'>, <Atom N9>, <Atom C8>, <Atom N7>, <Atom C5>, <Atom C6>, <Atom N6>, <Atom N1>, <Atom C2>, <Atom N3>, <Atom C4>]



These are all non-GLY residues without CB atom, so we are not able to calculate `HSExposureCB`.

### Pseudo-CB coordinates

In [16]:
# These should refer to GLY residues
hse_cb_coord = exposures_cb.ca_cb_list
hse_cb_coord[:3]

[(<Vector -8.03, 23.35, 20.43>, <Vector -7.41, 24.55, 19.89>),
 (<Vector -0.11, 16.57, 15.27>, <Vector 1.29, 16.93, 15.20>),
 (<Vector 3.05, 10.92, 16.51>, <Vector 4.32, 11.61, 16.65>)]

In [17]:
[residue for residue in residues if residue.get_resname() == 'GLY']

[<Residue GLY het=  resseq=136 icode= >,
 <Residue GLY het=  resseq=140 icode= >,
 <Residue GLY het=  resseq=142 icode= >,
 <Residue GLY het=  resseq=145 icode= >,
 <Residue GLY het=  resseq=173 icode= >,
 <Residue GLY het=  resseq=198 icode= >,
 <Residue GLY het=  resseq=216 icode= >,
 <Residue GLY het=  resseq=265 icode= >,
 <Residue GLY het=  resseq=268 icode= >,
 <Residue GLY het=  resseq=291 icode= >,
 <Residue GLY het=  resseq=303 icode= >,
 <Residue GLY het=  resseq=316 icode= >,
 <Residue GLY het=  resseq=325 icode= >,
 <Residue GLY het=  resseq=355 icode= >]

In [18]:
residue_ca_coord = [residue['CA'].get_vector() for residue in residues if residue.get_resname() == 'GLY']
residue_ca_coord[:3]

[<Vector -8.03, 23.35, 20.43>,
 <Vector -0.11, 16.57, 15.27>,
 <Vector 3.05, 10.92, 16.51>]

In [19]:
for i, j in zip(hse_cb_coord, residue_ca_coord):
    print(i[0].get_array() == j.get_array())

[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]
[ True  True  True]


## `HSExposureCA`

In [20]:
exposures_ca = HSExposureCA(chain, 13)

In [21]:
len(exposures_ca)

243

### Missing residues after calculation

In [22]:
residue_ids_hse_ca = [r[0].id[1] for r in exposures_ca.property_list]
missing_hse_ca = set(residue_ids) - set(residue_ids_hse_ca)
print(f'Residues with missing HSExposureCA calculation: {missing_hse_ca}')

Residues with missing HSExposureCA calculation: {291, 388, 275, 501, 127}


These are all residues at beginning, end or gap in chain, where we do not have three residues (CA) in a row to calculate `HSExposureCA`.

### Pseudo-CB coordinates

In [23]:
len(exposures_ca.ca_cb_list)

256

### Map residues to `HSExposureCA/B.ca_cb_list`

In [24]:
def _get_residues_list(exposures, method):
    
    if method == 'HSExposureCB':
        residues_list = [i[0] for i in exposures.property_list if i[0].get_resname() == 'GLY']
        
    elif method == 'HSExposureCA':
        residues_list = []
        for i in exposures.property_list:
            
            if i[0].get_resname() == 'GLY':  # If GLY add residue twice...
                residues_list.extend([i[0], i[0]])
            else:
                residues_list.append(i[0])
    
    else:
        pass
                
    return residues_list

In [25]:
def _map_residue2exposures(residues_list, exposures):
    
    hse_cb_coord = exposures.ca_cb_list
    
    if len(hse_cb_coord) != len(residues_list):
        raise ValueError(f'Bla.')
        
    data = []
    
    for i, j in zip(residues_list, hse_cb_coord):
        
        data.append(
            [
                i.id[1], 
                i.get_resname(), 
                i['CA'].get_vector(), 
                j[0], 
                j[1]]
        )
        
    data = pd.DataFrame(data, columns='residue_id residue_name residue_CA HSE_CA HSE_pCB'.split())
    data.set_index('residue_id', inplace=True)
    
    # Check if 
    CA_bool = []

    for index, row in data.iterrows():
        CA_bool.append(all(row.residue_CA.get_array() == row.HSE_CA.get_array()))
        
    if not all(CA_bool):
        raise ValueError(f'Not matching.')
        
    return data

In [26]:
def _get_pCB_coordinate(residue_id, exposures, method):
    
    residues_list = _get_residues_list(exposures, method)
    data = _map_residue2exposures(residues_list, exposures)
    
    if residue_id not in data.index:
        raise IndexError(f'Residue ID is no GLY.')
    
    return data.loc[residue_id, 'HSE_pCB']

In [27]:
data = _get_pCB_coordinate(136, exposures_ca, 'HSExposureCA')
data

residue_id
136    <Vector -8.38, 24.25, 20.18>
136    <Vector -7.41, 24.55, 19.89>
Name: HSE_pCB, dtype: object

In [28]:
data = _get_pCB_coordinate(136, exposures_cb, 'HSExposureCB')
data

<Vector -7.41, 24.55, 19.89>

## Use only selected residues?

In [29]:
from Bio.PDB.Chain import Chain

In [30]:
def _get_pcb_from_gly(residue):
    
    try:
        if residue.get_resname() != 'GLY':
            raise ValueError(f'Residue is not GLY, but {residue.get_resname()}.')
    except KeyError:
        raise ValueError(f'Residue is not GLY, but non-standard residues.')
    
    
    try:
        CA = residue['CA'].get_vector()
        pCB = HSExposureCB(Chain(id='X'))._get_gly_cb_vector(residue)
        CA_pCB = CA + pCB
        return CA_pCB
        
    except KeyError:
        return None

In [31]:
_get_pcb_from_gly(residues[9])

<Vector -7.41, 24.55, 19.89>

In [32]:
def _get_pcb_from_residue(residue, chain):
    """WIP"""
    
    try:
        if residue.get_resname() == 'GLY':
            raise ValueError(f'Residue is GLY, please GLY method instead.')
    except KeyError:
        pass
    
    try:
        CA = residue['CA'].get_vector()
        pCB = HSExposureCA(Chain(id='X'))._get_cb(residue1, residue2, residue3)
        CA_pCB = CA + pCB[0]
        return CA_pCB
        
    except KeyError:
        return None